Интенты:  
1. "Болталка" (FastText, датасет с урока №3 https://disk.yandex.ru/d/SLgpFrmrBubeYg).  
2. "Шеф повар" (предобученная модель huggingface sberbank-ai/rugpt3small_based_on_gpt2, finetuning на датасете https://www.kaggle.com/datasets/coolonce/recipes-and-interpretation-dim).  
3. Базовые интенты - "текущее время", "прощание".  
Для классификации интентов сначала проверяем наличие определенных слов ("время", "час", "спасибо", "пока"), далее классическая модель классификатора с TFIdf-векторайзером.

In [1]:
import numpy as np
import pandas as pd

from ufal.udpipe import Model, Pipeline
from gensim.models import FastText
import annoy

import wget
import os
import sys

import pickle
from tqdm.notebook import tqdm

## 1. Интент "Болталка"

### 1.1. Подготовка данных

#### Преобразование файла

In [2]:
%%time
question = None
written = False
with open('./data/prepared_answers.txt', 'w', encoding='utf-8') as fout:
    with open('./data/Otvety.txt', 'r', encoding='utf-8') as fin:
        for line in tqdm(fin):
            if line.startswith("---"):
                written = False
                continue
            if not written and question is not None:
                fout.write(question.replace("\t", " ").strip() + "\t" + line.replace("\t", " "))
                written = True
                question = None
                continue
            if not written:
                question = line.strip()
                continue

0it [00:00, ?it/s]

Wall time: 19.8 s


#### Предобработка текстов

https://github.com/akutuzov/webvectors/blob/master/preprocessing/rusvectores_tutorial.ipynb
https://github.com/akutuzov/webvectors/blob/master/preprocessing/rus_preprocessing_udpipe.py

In [3]:
def num_replace(word):
    newtoken = 'x' * len(word)
    return newtoken


def clean_token(token, misc):
    out_token = token.strip().replace(' ', '')
    if token == 'Файл' and 'SpaceAfter=No' in misc:
        return None
    return out_token


def clean_lemma(lemma, pos):
    out_lemma = lemma.strip().replace(' ', '').replace('_', '').lower()
    if '|' in out_lemma or out_lemma.endswith('.jpg') or out_lemma.endswith('.png'):
        return None
    if pos != 'PUNCT':
        if out_lemma.startswith('«') or out_lemma.startswith('»'):
            out_lemma = ''.join(out_lemma[1:])
        if out_lemma.endswith('«') or out_lemma.endswith('»'):
            out_lemma = ''.join(out_lemma[:-1])
        if out_lemma.endswith('!') or out_lemma.endswith('?') or out_lemma.endswith(',') \
                or out_lemma.endswith('.'):
            out_lemma = ''.join(out_lemma[:-1])
    return out_lemma

In [4]:
def process(pipeline, text='Строка', keep_pos=True, keep_punct=False):
    entities = {'PROPN'}
    named = False
    memory = []
    mem_case = None
    mem_number = None
    tagged_propn = []

    # обрабатываем текст, получаем результат в формате conllu:
    processed = pipeline.process(text)

    # пропускаем строки со служебной информацией:
    content = [l for l in processed.split('\n') if not l.startswith('#')]

    # извлекаем из обработанного текста леммы, тэги и морфологические характеристики
    tagged = [w.split('\t') for w in content if w]

    for t in tagged:
        if len(t) != 10:
            continue
        (word_id, token, lemma, pos, xpos, feats, head, deprel, deps, misc) = t
        token = clean_token(token, misc)
        lemma = clean_lemma(lemma, pos)
        if not lemma or not token:
            continue
        if pos in entities:
            if '|' not in feats:
                tagged_propn.append('%s_%s' % (lemma, pos))
                continue
            morph = {el.split('=')[0]: el.split('=')[1] for el in feats.split('|')}
            if 'Case' not in morph or 'Number' not in morph:
                tagged_propn.append('%s_%s' % (lemma, pos))
                continue
            if not named:
                named = True
                mem_case = morph['Case']
                mem_number = morph['Number']
            if morph['Case'] == mem_case and morph['Number'] == mem_number:
                memory.append(lemma)
                if 'SpacesAfter=\\n' in misc or 'SpacesAfter=\s\\n' in misc:
                    named = False
                    past_lemma = '::'.join(memory)
                    memory = []
                    tagged_propn.append(past_lemma + '_PROPN ')
            else:
                named = False
                past_lemma = '::'.join(memory)
                memory = []
                tagged_propn.append(past_lemma + '_PROPN ')
                tagged_propn.append('%s_%s' % (lemma, pos))
        else:
            if not named:
                if pos == 'NUM' and token.isdigit():  # Заменяем числа на xxxxx той же длины
                    lemma = num_replace(token)
                tagged_propn.append('%s_%s' % (lemma, pos))
            else:
                named = False
                past_lemma = '::'.join(memory)
                memory = []
                tagged_propn.append(past_lemma + '_PROPN ')
                tagged_propn.append('%s_%s' % (lemma, pos))

    if not keep_punct:
        tagged_propn = [word for word in tagged_propn if word.split('_')[1] != 'PUNCT']
    if not keep_pos:
        tagged_propn = [word.split('_')[0] for word in tagged_propn]
    return tagged_propn

In [5]:
udpipe_model_url = 'https://rusvectores.org/static/models/udpipe_syntagrus.model'
udpipe_filename = './models/' + udpipe_model_url.split('/')[-1]

if not os.path.isfile(udpipe_filename):
    print('UDPipe model not found. Downloading...', file=sys.stderr)
    wget.download(udpipe_model_url)

print('\nLoading the model...', file=sys.stderr)
model = Model.load(udpipe_filename)
process_pipeline = Pipeline(model, 'tokenize', Pipeline.DEFAULT, Pipeline.DEFAULT, 'conllu')


Loading the model...


In [6]:
def preprocess_text(txt, keep_pos):
    txt = str(txt)
    output = process(process_pipeline, text=txt, keep_pos=keep_pos)
    return output

In [7]:
%%time
sentences = []
c = 1
with open('./data/Otvety.txt', 'r', encoding='utf-8') as fin:
    for line in tqdm(fin):
        spls = preprocess_text(line, keep_pos=False)
        sentences.append(spls)
        c += 1
        if c > 500000:
            break

0it [00:00, ?it/s]

Wall time: 2h 20s


In [8]:
sentences = [i for i in sentences if len(i) > 2]

In [9]:
print(len(sentences))
sentences[1000]

378290


['в',
 'сколько',
 'раз',
 'волна',
 'длина',
 '0.4',
 'микрометр',
 'ослабевать',
 'вследствие',
 'молекулярный',
 'рассеивание',
 'чем',
 'волна',
 '0.75',
 'мкм']

#### Сохраним результат

In [10]:
with open('./data/sentences.pkl', 'wb') as f:
    pickle.dump(sentences, f)

 ### 1.2. Обучение модели FastText

In [11]:
%%time
print('Start train FastText...')
modelFT = FastText(sentences=sentences, vector_size=100, min_count=1, window=5, workers=8)

Start train FastText...
Wall time: 3min 16s


#### Сохраним результат

In [12]:
modelFT.save('./models/ft_model')

### 1.3. Построение индекса "ближайших соседей" и словаря маппинга

In [13]:
%%time
ft_index = annoy.AnnoyIndex(100 ,'angular')
index_map = {}
counter = 0
with open('./data/prepared_answers.txt', 'r', encoding='utf-8') as f:
    for line in tqdm(f):
        n_ft = 0
        spls = line.split("\t")
        index_map[counter] = spls[1]
        question = preprocess_text(spls[0], keep_pos=False)
        vector_ft = np.zeros(100)
        for word in question:
            if word in modelFT.wv:
                vector_ft += modelFT.wv[word]
                n_ft += 1
        if n_ft > 0:
            vector_ft = vector_ft / n_ft
        ft_index.add_item(counter, vector_ft)
            
        counter += 1

ft_index.build(10)

0it [00:00, ?it/s]

Wall time: 4h 47min 49s


True

#### Сохраним результаты

In [14]:
ft_index.save('./models/speaker.ann')
with open('./models/index_map.pkl', 'wb') as f:
    pickle.dump(index_map, f)

### 1.4. Загрузка сохранённых объектов (опционально)

In [15]:
if 0:
    #Предобработанные тексты
    with open('./data/sentences.pkl', 'rb') as f:
        sentences = pickle.load(f)
    
    #Модель FastText
    modelFT = FastText.load("./models/ft_model")
    
    #AnnoyIndex
    ft_index = annoy.AnnoyIndex(100, 'angular')
    ft_index.load('./models/speaker.ann') 
    
    #Маппинг
    with open('./models/index_map.pkl', 'rb') as f:
        index_map = pickle.load(f)

### 1.5. Тест "болталки"

In [16]:
def embed_txt(txt, idfs, midf):
    n_ft = 0
    vector_ft = np.zeros(100)
    for word in txt:
        if word in modelFT.wv:
            vector_ft += modelFT.wv[word] * 1 # idfs.get(word, midf)
            n_ft += 1 # idfs.get(word, midf)
    return vector_ft / n_ft if n_ft > 0 else vector_ft

In [17]:
def get_speaker_answer(input_txt, var_count):
    input_txt = preprocess_text(input_txt, keep_pos=False)
    vect_ft = embed_txt(input_txt, {}, 1)
    ft_index_val, distances = ft_index.get_nns_by_vector(vect_ft, var_count, include_distances=True)
    if distances[0] > 0.35:
        print("Моя твоя не понимать")
        print(distances[0])
        return
    print(index_map[np.random.choice(ft_index_val)])

In [18]:
get_speaker_answer('Как тебя зовут?', 1)

непринужденно так.... как будто приглашают..... 



## 2. Интент "Шеф-повар"

In [19]:
import re
from sklearn.model_selection import train_test_split

from transformers import AutoTokenizer
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments, AutoModelForCausalLM

### 2.1. Подготовка данных

Датасет https://www.kaggle.com/datasets/coolonce/recipes-and-interpretation-dim

In [20]:
df_rec = pd.read_csv('./data/all_recepies_inter.csv', sep='\t')

In [21]:
df_rec.head()

,Unnamed: 0,name,composition,cooking_type,Инструкции,dish_type,Дата,photo,source,composition_inter
0,0,рассольник классический с перловкой и солеными...,"[{'Перловка': 0.1, 'unit': 'стак. (200 мл)'}, ...","варка,жарка",Подготовить указанные ингредиенты для приготов...,первое,05.06.2015,photo_1000menu_1.jpg,https://1000.menu/cooking/33395-rassolnik-s-pe...,"[{'product_id': 4253, 'name_source': 'Перловая..."
1,1,Суп пюре из белокочаной капусты,"[{'Капуста белокочанная': 50.0, 'unit': 'гр'},...",варка,"Необходимые ингредиенты\r\nНарезаем лук, морко...",первое,27.06.2015,photo_1000menu_2.jpg,https://1000.menu/cooking/25399-sup-pure-iz-be...,"[{'product_id': 2286, 'name_source': 'Капуста ..."
2,2,Постные щи из квашеной капусты,"[{'Капуста квашеная': 116.7, 'unit': 'гр'}, {'...","варка,жарка,тушение","Честно признаюсь, у меня не было репы на момен...",первое,12.02.2013,photo_1000menu_3.jpg,https://1000.menu/cooking/5159-postnje-shchi,"[{'product_id': 0, 'name_source': 'Капуста ква..."
3,3,Тюря- простой суп быстро и вкусно,"[{'Квас': 0.2, 'unit': 'л'}, {'Лук репчатый': ...",сырое,"\r\nНачинаем мы приготовление тюри с того, что...",первое,02.03.2011,photo_1000menu_4.jpg,https://1000.menu/cooking/5085-turya,"[{'product_id': 0, 'name_source': 'Квас', 'uni..."
4,4,Фасолевый суп из красной фасоли,"[{'Вода': 0.3, 'unit': 'л'}, {'Картошка': 0.3,...",варка,Подготовить ингредиенты. Для приготовления суп...,первое,28.01.2013,photo_1000menu_5.jpg,https://1000.menu/cooking/38765-fasolevyi-sup-...,"[{'product_id': 828, 'name_source': 'Вода', 'u..."


In [22]:
df_rec.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27884 entries, 0 to 27883
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Unnamed: 0         27884 non-null  int64 
 1   name               27884 non-null  object
 2   composition        27884 non-null  object
 3   cooking_type       27884 non-null  object
 4   Инструкции         27884 non-null  object
 5   dish_type          27884 non-null  object
 6   Дата               27884 non-null  object
 7   photo              27884 non-null  object
 8   source             27884 non-null  object
 9   composition_inter  27884 non-null  object
dtypes: int64(1), object(9)
memory usage: 2.1+ MB


In [23]:
data = df_rec.loc[:10000, 'Инструкции']

In [24]:
def build_text_files(data_json, dest_path):
    f = open(dest_path, 'w', encoding='utf-8')
    data = ''
    for texts in data_json:
        summary = str(texts).strip()
        summary = re.sub(r"\s", " ", summary)
        data += summary + "  "
    f.write(data)

In [25]:
train, test = train_test_split(data, test_size=0.2)

build_text_files(train,'./data/train_dataset.txt')
build_text_files(test,'./data/test_dataset.txt')

### 2.2. Токенайзер, пайплайн данных, модель

In [26]:
tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [27]:
def load_dataset(train_path, test_path, tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path='./data/train_dataset.txt',
          block_size=128)

    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path='./data/test_dataset.txt',
          block_size=128)

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset, test_dataset, data_collator

In [28]:
train_dataset, test_dataset, data_collator = load_dataset('./data/train_dataset.txt',
                                                          './data/test_dataset.txt',
                                                          tokenizer)

C:\Users\Kartsev.ES\Anaconda3\lib\site-packages\transformers\data\datasets\language_modeling.py:54: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [29]:
model = AutoModelForCausalLM.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")

### 2.3. Fine-tuning модели

In [30]:
training_args = TrainingArguments(
    output_dir="./models/gpt2-chief", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=3, # number of training epochs
    per_device_train_batch_size=4, # batch size for training
    per_device_eval_batch_size=4,  # batch size for evaluation
    eval_steps = 400, # Number of update steps between two evaluations.
    save_steps=800, # after # steps model is saved
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    )

In [31]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

In [32]:
trainer.train()

C:\Users\Kartsev.ES\Anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 12207
  Num Epochs = 3
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 9156


Step,Training Loss
500,2.505600
1000,2.414500
1500,2.370700
2000,2.317800
2500,2.273900
3000,2.255900
3500,2.119800
4000,2.095200
4500,2.078400
5000,2.079800


Saving model checkpoint to ./models/gpt2-chief\checkpoint-800
Configuration saved in ./models/gpt2-chief\checkpoint-800\config.json
Model weights saved in ./models/gpt2-chief\checkpoint-800\pytorch_model.bin
Saving model checkpoint to ./models/gpt2-chief\checkpoint-1600
Configuration saved in ./models/gpt2-chief\checkpoint-1600\config.json
Model weights saved in ./models/gpt2-chief\checkpoint-1600\pytorch_model.bin
Saving model checkpoint to ./models/gpt2-chief\checkpoint-2400
Configuration saved in ./models/gpt2-chief\checkpoint-2400\config.json
Model weights saved in ./models/gpt2-chief\checkpoint-2400\pytorch_model.bin
Saving model checkpoint to ./models/gpt2-chief\checkpoint-3200
Configuration saved in ./models/gpt2-chief\checkpoint-3200\config.json
Model weights saved in ./models/gpt2-chief\checkpoint-3200\pytorch_model.bin
Saving model checkpoint to ./models/gpt2-chief\checkpoint-4000
Configuration saved in ./models/gpt2-chief\checkpoint-4000\config.json
Model weights saved in ./

TrainOutput(global_step=9156, training_loss=2.1218886102532237, metrics={'train_runtime': 35271.0296, 'train_samples_per_second': 1.038, 'train_steps_per_second': 0.26, 'total_flos': 2392193875968000.0, 'train_loss': 2.1218886102532237, 'epoch': 3.0})

#### Сохраним результаты

In [33]:
tokenizer.save_pretrained('./models/gpt_chf')
model.save_pretrained('./models/model_gpt_chf')

tokenizer config file saved in ./models/gpt_chf\tokenizer_config.json
Special tokens file saved in ./models/gpt_chf\special_tokens_map.json
Configuration saved in ./models/model_gpt_chf\config.json
Model weights saved in ./models/model_gpt_chf\pytorch_model.bin


### 2.4. Загрузка сохранённых объектов (опционально)

In [34]:
if 0:
    tokenizer = AutoTokenizer.from_pretrained('./models/gpt_chf')
    model = AutoModelForCausalLM.from_pretrained('./models/model_gpt_chf')

### 2.5. Тест модели

In [35]:
def gpt_generate(inp_txt):
    tokens = tokenizer(inp_txt, return_tensors='pt')
    size = tokens['input_ids'].shape[1]
    output = model.generate(
        **tokens,
        #end_token=end_token_id,
        do_sample=False,
        max_length=size+100,
        repetition_penalty=5.,
        temperature=0.5,
        num_beams=2,
    )
    
    decoded = tokenizer.decode(output[0])
    result = decoded[len(inp_txt):]
    return result

In [36]:
inp_txt = 'Что сделать из рыбы?'
print(inp_txt)
print(gpt_generate(inp_txt))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Что сделать из рыбы?
  2. Натереть рыбу солью и перцем, положить в кастрюлю с кипящей подсоленной водой на 1–1,5 часа (время зависит от размера рыбы).  3. Рыба должна быть не очень мягкой, но и не слишком прожаренной.  4. Когда рыба будет готова, вынуть ее шумовкой из кастрюли и дать немного остыть.  5. Разогреть духовку до 180 градусов.  6. В сковороде разогреть оливковое масло


## 3. Классификация интентов

### 3.1. Подготовка данных

In [37]:
%%time
data_prep = data[:5000].apply(preprocess_text, keep_pos=False)
data_prep.head()

Wall time: 6min 10s


0    [подготавливать, указывать, ингредиент, для, п...
1    [необходимый, ингредиент, нарезать, лук, морко...
2    [честно, признаваться, у, я, не, быть, репа, н...
3    [начинать, мы, приготовление, тюря, с, то, что...
4    [подготавливать, ингредиент, для, приготовлени...
Name: Инструкции, dtype: object

In [38]:
positive_texts = [" ".join(val) for val in data_prep.values]

In [39]:
positive_texts[0]

'подготавливать указывать ингредиент для приготовление рассольник с перловый крупа мясной бульон сварить заранее из говядина или из курица также можно сварить и вегетарианский суп на вода обычно я рассольник варить без томатный паста но тут для разнообразие решать добавлять это по желание из специя соль черный перец горошек душистый перец перловый крупа промывать до чистый вода в горячий бульон добавлять промытой перловка и варить на среднее огонь для рассольник хорошо брать кислый очень соленый огурец если же огурец обычный то рекомендовать в рассольник добавлять из сам рассол от огурец соленый огурец доставать из рассол и натереть на крупный терка картофель помыть обсушить очищать нарезать кубик пока очередь картофель не подходить класть он в вода морковь лук чеснок очищать морковь натереть на крупный терка лук сельдерей чеснок мелко порезать обжарить в масло овощ добавлять томатный паста томатный паста по желание минута через двадцать добавлять к перловый крупа нарезать картофель и 

In [40]:
negative_samples = np.random.choice(np.array(sentences, dtype=object), size=len(positive_texts), replace=False)

In [41]:
negative_texts = [" ".join(sentence) for sentence in negative_samples]

In [42]:
negative_texts[0]

'я бы просто идиот называть судить по вопросамя'

In [43]:
dataset = negative_texts + positive_texts
labels = np.zeros(len(dataset))
labels[len(negative_texts):] = np.ones(len(positive_texts))

In [44]:
X_train, X_test, y_train, y_test = train_test_split(dataset, labels, test_size=0.2, stratify=labels,
                                                    random_state=42)

In [45]:
X_train[:2]

['за что сильный все в мир можно полюбить человек не любить же кто-то исключительно изз внешность или изз деньги или изз он помощи даже изз он ум тогда за что за что можно любить сильно все на свет',
 'x нарезать капуста картофель и бросать вариться x нарезать лук и натереть на терка морковь x когда вода с капуста и картофель закипеть добавлять в кастрюля лук с морковью x специя зелень и масло добавлять когда овощ все свариться поварить пара минута и снимать с огонь']

In [46]:
np.sum(y_train)

4000.0

In [47]:
len(y_train)

8000

### 3.2. Обучение модели классификатора

In [48]:
from sklearn.feature_extraction.text import TfidfVectorizer
from catboost import CatBoostClassifier

In [50]:
vectorizer = TfidfVectorizer(ngram_range=(1, 2))
classifier = CatBoostClassifier()

x_train_vec = vectorizer.fit_transform(X_train)
x_test_vec = vectorizer.transform(X_test)

classifier.fit(x_train_vec, y_train)

Learning rate set to 0.025035
0:	learn: 0.6437870	total: 1.11s	remaining: 18m 34s
1:	learn: 0.5998836	total: 2.18s	remaining: 18m 10s
2:	learn: 0.5609188	total: 3.08s	remaining: 17m 5s
3:	learn: 0.5225950	total: 3.96s	remaining: 16m 26s
4:	learn: 0.4876022	total: 4.71s	remaining: 15m 37s
5:	learn: 0.4558181	total: 5.44s	remaining: 15m
6:	learn: 0.4281055	total: 6.15s	remaining: 14m 32s
7:	learn: 0.3999046	total: 6.88s	remaining: 14m 13s
8:	learn: 0.3747137	total: 7.61s	remaining: 13m 58s
9:	learn: 0.3513087	total: 8.33s	remaining: 13m 44s
10:	learn: 0.3305317	total: 9.05s	remaining: 13m 34s
11:	learn: 0.3120839	total: 9.75s	remaining: 13m 22s
12:	learn: 0.2939754	total: 10.5s	remaining: 13m 14s
13:	learn: 0.2770522	total: 11.2s	remaining: 13m 6s
14:	learn: 0.2617006	total: 11.9s	remaining: 13m 1s
15:	learn: 0.2488514	total: 12.6s	remaining: 12m 55s
16:	learn: 0.2354862	total: 13.3s	remaining: 12m 50s
17:	learn: 0.2245671	total: 14.1s	remaining: 12m 50s
18:	learn: 0.2133910	total: 14.9s

153:	learn: 0.0327616	total: 1m 55s	remaining: 10m 35s
154:	learn: 0.0325805	total: 1m 56s	remaining: 10m 34s
155:	learn: 0.0325532	total: 1m 57s	remaining: 10m 33s
156:	learn: 0.0325187	total: 1m 57s	remaining: 10m 31s
157:	learn: 0.0324955	total: 1m 58s	remaining: 10m 31s
158:	learn: 0.0324712	total: 1m 59s	remaining: 10m 30s
159:	learn: 0.0324459	total: 1m 59s	remaining: 10m 29s
160:	learn: 0.0324151	total: 2m	remaining: 10m 28s
161:	learn: 0.0323925	total: 2m 1s	remaining: 10m 27s
162:	learn: 0.0323647	total: 2m 1s	remaining: 10m 26s
163:	learn: 0.0323268	total: 2m 2s	remaining: 10m 25s
164:	learn: 0.0319227	total: 2m 3s	remaining: 10m 24s
165:	learn: 0.0319008	total: 2m 4s	remaining: 10m 23s
166:	learn: 0.0318761	total: 2m 4s	remaining: 10m 22s
167:	learn: 0.0314997	total: 2m 5s	remaining: 10m 21s
168:	learn: 0.0314753	total: 2m 6s	remaining: 10m 20s
169:	learn: 0.0312656	total: 2m 6s	remaining: 10m 19s
170:	learn: 0.0312206	total: 2m 7s	remaining: 10m 18s
171:	learn: 0.0307801	to

306:	learn: 0.0231348	total: 3m 48s	remaining: 8m 36s
307:	learn: 0.0231211	total: 3m 49s	remaining: 8m 35s
308:	learn: 0.0231069	total: 3m 50s	remaining: 8m 34s
309:	learn: 0.0230945	total: 3m 50s	remaining: 8m 34s
310:	learn: 0.0230787	total: 3m 51s	remaining: 8m 33s
311:	learn: 0.0229681	total: 3m 52s	remaining: 8m 32s
312:	learn: 0.0229488	total: 3m 53s	remaining: 8m 31s
313:	learn: 0.0226783	total: 3m 53s	remaining: 8m 30s
314:	learn: 0.0226653	total: 3m 54s	remaining: 8m 30s
315:	learn: 0.0226513	total: 3m 55s	remaining: 8m 29s
316:	learn: 0.0226406	total: 3m 55s	remaining: 8m 28s
317:	learn: 0.0226283	total: 3m 56s	remaining: 8m 27s
318:	learn: 0.0226191	total: 3m 57s	remaining: 8m 26s
319:	learn: 0.0226072	total: 3m 58s	remaining: 8m 25s
320:	learn: 0.0225932	total: 3m 58s	remaining: 8m 25s
321:	learn: 0.0225840	total: 3m 59s	remaining: 8m 24s
322:	learn: 0.0225737	total: 4m	remaining: 8m 23s
323:	learn: 0.0225640	total: 4m 1s	remaining: 8m 23s
324:	learn: 0.0223342	total: 4m 1

460:	learn: 0.0149361	total: 5m 40s	remaining: 6m 37s
461:	learn: 0.0149235	total: 5m 40s	remaining: 6m 37s
462:	learn: 0.0149176	total: 5m 41s	remaining: 6m 36s
463:	learn: 0.0149105	total: 5m 42s	remaining: 6m 35s
464:	learn: 0.0148966	total: 5m 43s	remaining: 6m 35s
465:	learn: 0.0148846	total: 5m 44s	remaining: 6m 34s
466:	learn: 0.0148653	total: 5m 45s	remaining: 6m 33s
467:	learn: 0.0147998	total: 5m 45s	remaining: 6m 33s
468:	learn: 0.0147872	total: 5m 46s	remaining: 6m 32s
469:	learn: 0.0147813	total: 5m 47s	remaining: 6m 31s
470:	learn: 0.0146120	total: 5m 47s	remaining: 6m 30s
471:	learn: 0.0146016	total: 5m 48s	remaining: 6m 29s
472:	learn: 0.0145950	total: 5m 49s	remaining: 6m 29s
473:	learn: 0.0145891	total: 5m 49s	remaining: 6m 28s
474:	learn: 0.0145836	total: 5m 50s	remaining: 6m 27s
475:	learn: 0.0144869	total: 5m 51s	remaining: 6m 26s
476:	learn: 0.0144767	total: 5m 52s	remaining: 6m 26s
477:	learn: 0.0143542	total: 5m 52s	remaining: 6m 25s
478:	learn: 0.0143486	total:

613:	learn: 0.0111124	total: 7m 31s	remaining: 4m 43s
614:	learn: 0.0111045	total: 7m 32s	remaining: 4m 43s
615:	learn: 0.0111010	total: 7m 32s	remaining: 4m 42s
616:	learn: 0.0110975	total: 7m 33s	remaining: 4m 41s
617:	learn: 0.0110929	total: 7m 34s	remaining: 4m 40s
618:	learn: 0.0110894	total: 7m 35s	remaining: 4m 40s
619:	learn: 0.0110524	total: 7m 35s	remaining: 4m 39s
620:	learn: 0.0109356	total: 7m 36s	remaining: 4m 38s
621:	learn: 0.0109288	total: 7m 37s	remaining: 4m 37s
622:	learn: 0.0109206	total: 7m 37s	remaining: 4m 37s
623:	learn: 0.0109171	total: 7m 38s	remaining: 4m 36s
624:	learn: 0.0108984	total: 7m 39s	remaining: 4m 35s
625:	learn: 0.0108912	total: 7m 40s	remaining: 4m 34s
626:	learn: 0.0108839	total: 7m 41s	remaining: 4m 34s
627:	learn: 0.0108545	total: 7m 41s	remaining: 4m 33s
628:	learn: 0.0108470	total: 7m 42s	remaining: 4m 32s
629:	learn: 0.0108394	total: 7m 43s	remaining: 4m 32s
630:	learn: 0.0108360	total: 7m 44s	remaining: 4m 31s
631:	learn: 0.0107558	total:

767:	learn: 0.0086997	total: 9m 23s	remaining: 2m 50s
768:	learn: 0.0086975	total: 9m 24s	remaining: 2m 49s
769:	learn: 0.0086951	total: 9m 25s	remaining: 2m 48s
770:	learn: 0.0086928	total: 9m 26s	remaining: 2m 48s
771:	learn: 0.0086905	total: 9m 26s	remaining: 2m 47s
772:	learn: 0.0086857	total: 9m 27s	remaining: 2m 46s
773:	learn: 0.0086681	total: 9m 28s	remaining: 2m 45s
774:	learn: 0.0086616	total: 9m 29s	remaining: 2m 45s
775:	learn: 0.0086144	total: 9m 30s	remaining: 2m 44s
776:	learn: 0.0086113	total: 9m 31s	remaining: 2m 43s
777:	learn: 0.0086090	total: 9m 31s	remaining: 2m 43s
778:	learn: 0.0086072	total: 9m 32s	remaining: 2m 42s
779:	learn: 0.0086050	total: 9m 33s	remaining: 2m 41s
780:	learn: 0.0085771	total: 9m 33s	remaining: 2m 40s
781:	learn: 0.0085304	total: 9m 34s	remaining: 2m 40s
782:	learn: 0.0085262	total: 9m 35s	remaining: 2m 39s
783:	learn: 0.0085241	total: 9m 36s	remaining: 2m 38s
784:	learn: 0.0085218	total: 9m 37s	remaining: 2m 38s
785:	learn: 0.0085178	total:

919:	learn: 0.0074676	total: 11m 14s	remaining: 58.7s
920:	learn: 0.0074659	total: 11m 15s	remaining: 57.9s
921:	learn: 0.0074133	total: 11m 15s	remaining: 57.2s
922:	learn: 0.0074116	total: 11m 16s	remaining: 56.4s
923:	learn: 0.0074097	total: 11m 17s	remaining: 55.7s
924:	learn: 0.0074080	total: 11m 18s	remaining: 55s
925:	learn: 0.0073903	total: 11m 18s	remaining: 54.2s
926:	learn: 0.0073854	total: 11m 19s	remaining: 53.5s
927:	learn: 0.0073821	total: 11m 20s	remaining: 52.8s
928:	learn: 0.0073796	total: 11m 20s	remaining: 52s
929:	learn: 0.0073364	total: 11m 21s	remaining: 51.3s
930:	learn: 0.0073206	total: 11m 22s	remaining: 50.6s
931:	learn: 0.0073180	total: 11m 23s	remaining: 49.8s
932:	learn: 0.0073162	total: 11m 23s	remaining: 49.1s
933:	learn: 0.0073145	total: 11m 24s	remaining: 48.4s
934:	learn: 0.0072901	total: 11m 25s	remaining: 47.6s
935:	learn: 0.0072901	total: 11m 25s	remaining: 46.9s
936:	learn: 0.0072884	total: 11m 26s	remaining: 46.2s
937:	learn: 0.0072845	total: 11m

In [51]:
from sklearn.metrics import accuracy_score

In [52]:
accuracy_score(y_true=y_test, y_pred=classifier.predict(x_test_vec))

0.9975

#### Сохраним результаты

In [53]:
with open('./models/classifier_cb.pkl', 'wb') as f:
    pickle.dump(classifier, f)

### 3.3. Загрузка сохранённых объектов (опционально)

In [54]:
if 0:
    with open('./models/classifier_cb.pkl', 'rb') as f:
        classifier = pickle.load(f)

## 4. Подключение чат-бота

### 4.1 Добавление базовых интентов

In [55]:
import datetime

In [56]:
set_1 = set(['спасибо', 'пока'])
set_2 = set(['час', 'время'])

In [57]:
def check_base_intent(input_txt):
    if set_1.intersection(set(input_txt)):
        return 'Обращайтесь, всегда готов помочь!'
    elif set_2.intersection(set(input_txt)):
        return datetime.datetime.now().strftime("%d-%m-%Y %H:%M")
    else:
        return None

### 4.2. Проверка работы функции

In [58]:
def get_speaker_answer(input_txt, var_count=1):
    input_txt = preprocess_text(input_txt, keep_pos=False)
    vect_ft = embed_txt(input_txt, {}, 1)
    ft_index_val, distances = ft_index.get_nns_by_vector(vect_ft, var_count, include_distances=True)
    if distances[0] > 0.35:
        return "Моя твоя не понимать"
    return index_map[np.random.choice(ft_index_val)]

In [59]:
def textMessage(text):
    input_txt = preprocess_text(text, keep_pos=False)
    print(f'Предобработанная последовательность:\n{input_txt}')
    
    base_answer = check_base_intent(input_txt)
    if base_answer:
        print(f'Базовый интент')
        print(f'{base_answer}')
        return
    
    vect = vectorizer.transform([" ".join(input_txt)])
    prediction = classifier.predict(vect)
    predict_pr = classifier.predict_proba(vect)
    print(f'Предсказание классификатора:\n{prediction}')
    print(f'Вероятности:\n{predict_pr}')
    print(f'Ответ чат-бота:')
    if prediction == 0:
        print(f'{get_speaker_answer(text)}')
        return

    gpt_generate(text)
    return

In [60]:
textMessage('Где можно отлично отдохнуть?')

Предобработанная последовательность:
['где', 'можно', 'отлично', 'отдыхать']
Предсказание классификатора:
[0.]
Вероятности:
[[0.99665022 0.00334978]]
Ответ чат-бота:
на хорошей работе !!!. 



In [61]:
textMessage('Хочется салат из помидор с сыром, есть рецепт?')

Предобработанная последовательность:
['хотеться', 'салат', 'из', 'помидор', 'с', 'сыр', 'быть', 'рецепт']
Предсказание классификатора:
[0.]
Вероятности:
[[0.99665022 0.00334978]]
Ответ чат-бота:
Для приготовления пирога из лаваша с мясом, сыром и помидорами черри обжарьте 500 грамм любого фарша, можно по вкусу добавить лук или чеснок. Смажьте форму для пирога сливочным маслом. Выложите на дно лаваш, так, чтобы его края заходили на бортики формы. Если лаваш тонкий, можно положить несколько слоев. Затем на лаваш выложите обжаренный фарш, сверху порезанные на половинки помидоры черри. Посыпьте пирог тертым твердым сыром и отправьте запекаться на 15-20 минут в духовку, на температуре 180 градусов Цельсия.. 



In [62]:
textMessage('Беру сахар, масло, муку, яйца. Как испечь торт?')

Предобработанная последовательность:
['брать', 'сахар', 'масло', 'мука', 'яйцо', 'как', 'испекать', 'торт']
Предсказание классификатора:
[0.]
Вероятности:
[[0.99665022 0.00334978]]
Ответ чат-бота:
Моя твоя не понимать


In [63]:
textMessage('Сколько время?')

Предобработанная последовательность:
['сколько', 'время']
Базовый интент
08-08-2022 17:44


In [64]:
textMessage('Который час?')

Предобработанная последовательность:
['который', 'час']
Базовый интент
08-08-2022 17:44


In [65]:
textMessage('Спасибо, будь здоров')

Предобработанная последовательность:
['спасибо', 'быть', 'здоровый']
Базовый интент
Обращайтесь, всегда готов помочь!


Похоже в примерах текстов, на которых обучалась "болталка", есть немало рецептов, или текстов о продуктах. Потому классификатор не очень хорошо детектирует интент шеф-повара.

### 4.3. Чат-бот

Технической возможности запуска бота к сожалению нет, согласно материалам консультаций должно работать как-то так:

In [66]:
import os
import logging
from telegram import Update
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext

In [67]:
updater = Updater(token='1058946299:AAFn0ESsBoqdzscvF-tYo0wcZvAUS_iJMCw') # Токен API к Telegram
dispatcher = updater.dispatcher

In [68]:
def startCommand(update: Update, context: CallbackContext):
    update.message.reply_text('Привет! Я бот шеф-повар. Расскажу рецепт, могу просто поболтать ни о чём.')

def textMessage(update: Update, context: CallbackContext):
    input_txt = preprocess_text(update.message.text, keep_pos=False)
    
    base_answer = check_base_intent(input_txt)
    if base_answer:
        update.message.reply_text(base_answer)
        return
    
    vect = vectorizer.transform([" ".join(input_txt)])
    prediction = classifier.predict(vect)
    if prediction == 0:
        update.message.reply_text(get_speaker_answer(text))
        return

    update.message.reply_text(gpt_generate(update.message.text))
    return

In [69]:
start_command_handler = CommandHandler('start', startCommand)
text_message_handler = MessageHandler(Filters.text, textMessage)
dispatcher.add_handler(start_command_handler)
dispatcher.add_handler(text_message_handler)
updater.start_polling(clean=True)
updater.idle()

C:\Users\Kartsev.ES\AppData\Local\Temp/ipykernel_48336/2364210538.py:5: TelegramDeprecationWarning: The argument `clean` of `start_polling` is deprecated. Please use `drop_pending_updates` instead.
  updater.start_polling(clean=True)


NetworkError: urllib3 HTTPError HTTPSConnectionPool(host='api.telegram.org', port=443): Max retries exceeded with url: /bot1058946299:AAFn0ESsBoqdzscvF-tYo0wcZvAUS_iJMCw/getMe (Caused by ConnectTimeoutError(<telegram.vendor.ptb_urllib3.urllib3.connection.VerifiedHTTPSConnection object at 0x000001F1839686A0>, 'Connection to api.telegram.org timed out. (connect timeout=5.0)'))